# Chapter 05. 트리 알고리즘

## 05-3. 트리의 앙상블

## 앙상블 학습 Ensemble Learning
- 정형 데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘 <br>
  대부분 트리 모형을 기반으로 구성됨 <br><br>
- 앙상블 알고리즘 종류 <br>
  1) 랜덤 포레스트 <br>
  2) 엑스트라 트리 <br>
  3) 그레이디언트 부스팅 <br>
  4) 히스토그램 기반 그레이디언트 부스팅

### 랜덤 포레스트 Random Forest
- 결정 트리를 랜덤하게 만들어 (부트스트랩 샘플) 숲을 만들어내는 알고리즘 <br>
  : 각 결정 트리의 예측을 사용하여 최종 예측 <br>
  : 전체 특성 중 일부 특성을 무작위로 고린 다음 이 중에서 최선의 분할을 찾음 <br>
  : 무작위로 선택하여 훈련하기 때문에 과대적합을 방지하고 일반화 성능이 높다.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()

In [5]:
target = wine['class'].to_numpy()

In [6]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

#### 교차검증

In [7]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

In [8]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

In [9]:
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [10]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)   

[0.23167441 0.50039841 0.26792718]


#### OOB 샘플
- 부트스트랩 샘플에 포함되지 않고 남는 샘플 <br>
  : OOB 샘플을 이용하여 훈련한다. = 검증 세트의 역할

In [11]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)  

0.8934000384837406


### 엑스트라 트리 Extra Tree
- 랜덤 포레스트와 매우 비슷하지만 엑스트라 트리는 부트스트랩 샘플을 이용하지 않는다. <br>
  즉 각 결정 트리를 만들 때 전체 트리를 사용한다. <br>
  노드 분할도 최적의 분할이 아닌 무작위 분할을 한다.
- splitter = 'random'

In [12]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [13]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 

### 그레이디언트 부스팅
- 깊이가 얕은 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 <br>
  깊이가 얕은 트리 : 일반화 성능에 강하고 과대적합을 방지해준다.
- 기본적으로 깊이가 3인 결정트리 100개 사용

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [15]:
# 결정 트리의 개수를 500개로 늘리기
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [16]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68011572 0.16116151]


### 히스토그램 기반 그레이디언트 부스팅
- 입력특성을 256 구간으로 나누면서 시작하는 앙상블 <br>
  so, 노드 분할 시 최적의 분할을 빠르게 찾음
- 입력에 누락값이 있더리도 전처리 필요 없음
- 트리 개수 지정 매개변수는 n_estimators 아니고 max_iter
<br><br>
- 대표적인 히스토그램 기반 그레이디언트 부스팅 앙상블 라이브러리 : XGBoost, LightGBM 

In [17]:
from sklearn.experimental import enable_hist_gradient_boosting

In [18]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [19]:
from sklearn.inspection import permutation_importance

In [20]:
hgb.fit(train_input, train_target)

HistGradientBoostingClassifier(random_state=42)

#### n_repeats : 랜덤하게 섞을 횟수

In [21]:
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, n_jobs=-1)
print(result.importances_mean)

[0.08943621 0.23715605 0.07993073]


In [22]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [23]:
from xgboost import XGBClassifier

In [24]:
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

C:\Users\haeun\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:04:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\haeun\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:04:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\haeun\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:04:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\haeun\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:04:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\haeun\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:04:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9555033709953124 0.8799326275264677


#### LightGBM

In [25]:
from lightgbm import LGBMClassifier

In [26]:
lgb = LGBMClassifier(random_state=42)

In [27]:
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

In [28]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
